# Question 1

In [6]:
import pandas as pd
import numpy as np
import requests
from io import StringIO


pd.set_option("display.float_format", "{:,.2f}".format)   # pretty printing

In [7]:

def get_withdrawn() -> pd.DataFrame:
    """
    Returns a DataFrame containing the withdrawn IPOs.
    """
    url = f"https://stockanalysis.com/ipos/withdrawn/"
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/58.0.3029.110 Safari/537.3'
        )
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        # Wrap HTML text in StringIO to avoid deprecation warning
        # "Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object."
        html_io = StringIO(response.text)
        tables = pd.read_html(html_io)

        if not tables:
            raise ValueError(f"No tables found for year {year}.")

        return tables[0]

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except ValueError as ve:
        print(f"Data error: {ve}")
    except Exception as ex:
        print(f"Unexpected error: {ex}")

    return pd.DataFrame()

In [10]:

withdrawn_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Symbol          100 non-null    object
 1   Company Name    100 non-null    object
 2   Price Range     100 non-null    object
 3   Shares Offered  100 non-null    object
dtypes: object(4)
memory usage: 3.3+ KB


In [9]:

withdrawn_df = get_withdrawn()
withdrawn_df.head(10)

,Symbol,Company Name,Price Range,Shares Offered
0,ODTX,"Odyssey Therapeutics, Inc.",-,-
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000
2,AURN,"Aurion Biotech, Inc.",-,-
3,ROTR,"PHI Group, Inc.",-,-
4,ONE,One Power Company,-,-
5,HPOT,The Great Restaurant Development Holdings Limited,$4.00 - $6.00,1400000
6,CABR,"Caring Brands, Inc.",$4.00,750000
7,SQVI,"Sequoia Vaccines, Inc.",$8.00 - $10.00,2775000
8,SNI,Shenni Holdings Limited,$4.00 - $6.00,3000000
9,KMCM,Key Mining Corp.,$2.25,4444444


In [ ]:

# Shares Offered  → numeric
withdrawn_df["Shares Offered"] = (
    withdrawn_df["Shares Offered"]
        .astype(str)
        .str.replace(",", "", regex=False)
        .replace({"–": np.nan, "-": np.nan, "—": np.nan, "": np.nan})
        .astype(float)
)

# Ensure Price Range is object (string) for parsing
withdrawn_df["Price Range"] = withdrawn_df["Price Range"].astype(str)

withdrawn_df




,Symbol,Company Name,Price Range,Shares Offered
0,ODTX,"Odyssey Therapeutics, Inc.",-,NaN
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,"2,000,000.00"
2,AURN,"Aurion Biotech, Inc.",-,NaN
3,ROTR,"PHI Group, Inc.",-,NaN
4,ONE,One Power Company,-,NaN
...,...,...,...,...
95,FHP,"Freehold Properties, Inc.",-,NaN
96,CHO,Chobani Inc.,-,NaN
97,IFIT,iFIT Health & Fitness Inc.,$18.00 - $21.00,"30,769,231.00"
98,GLGX,"Gerson Lehrman Group, Inc.",-,NaN


In [15]:


def classify_company(name: str) -> str:
    """
    Map company names to classes in *priority* order:
    1) Acquisition Corp(/oration)  → Acq.Corp
    2) Inc(orporated)              → Inc
    3) Group                       → Group
    4) Ltd/Limited                 → Limited
    5) Holdings                    → Holdings
    else                           → Other
    """
    n = name.lower()
    # use word boundaries so 'inc' doesn't match 'income'
    patterns = [
        (r"\bacquisition corp(oration)?\b", "Acq.Corp"),
        (r"\binc(?:\.|orporated)?\b",        "Inc"),
        (r"\bgroup\b",                       "Group"),
        (r"\b(ltd|limited)\b",               "Limited"),
        (r"\bholdings?\b",                   "Holdings"),
    ]
    for pat, cls in patterns:
        if re.search(pat, n):
            return cls
    return "Other"

withdrawn_df["Company Class"] = withdrawn_df["Company Name"].apply(classify_company)
withdrawn_df["Company Class"].value_counts()


Company Class
Inc         51
Acq.Corp    21
Limited     17
Other        6
Group        4
Holdings     1
Name: count, dtype: int64

In [ ]:


# 
def price_range_to_avg(price_range):
    if pd.isna(price_range):
        return np.nan
    try:
        parts = str(price_range).replace('$', '').split('-')
        if len(parts) == 2:
            low, high = map(float, parts)
            return (low + high) / 2
        else:
            return float(parts[0])
    except Exception:
        return np.nan

withdrawn_df['Avg Price'] = withdrawn_df['Price Range'].apply(price_range_to_avg)

withdrawn_df


,Symbol,Company Name,Price Range,Shares Offered,Company Class,Avg Price
26,NVL,Novelis Inc.,$18.00 - $21.00,"45,000,000.00",Inc,19.50
97,IFIT,iFIT Health & Fitness Inc.,$18.00 - $21.00,"30,769,231.00",Inc,19.50
99,HCG,hear.com N.V.,$17.00 - $20.00,"16,220,000.00",Other,18.50
41,GGL,Games Global Limited,$16.00 - $19.00,"14,500,000.00",Limited,17.50
52,TFG,"The Fortegra Group, Inc.",$15.00 - $18.00,"18,000,000.00",Inc,16.50
...,...,...,...,...,...,...
98,GLGX,"Gerson Lehrman Group, Inc.",-,NaN,Inc,NaN
49,COFO,Coforge Limited,-,NaN,Limited,NaN
54,VNG,VNG Limited,-,"21,687,082.00",Limited,NaN
71,PICS,PicS Ltd.,-,NaN,Limited,NaN


In [34]:
withdrawn_df[["Price Range", "Avg Price"]].head(6)

,Price Range,Avg Price
26,$18.00 - $21.00,19.50
97,$18.00 - $21.00,19.50
99,$17.00 - $20.00,18.50
41,$16.00 - $19.00,17.50
52,$15.00 - $18.00,16.50
86,$13.00 - $15.00,14.00


In [44]:


# Then convert to numeric with coercion
withdrawn_df['Shares Offered'] = pd.to_numeric(withdrawn_df['Shares Offered'], errors='coerce')

# Ensure the dtype is float64 explicitly
withdrawn_df['Shares Offered'] = withdrawn_df['Shares Offered'].astype('float64')


withdrawn_df["Withdrawn Value"] = (
    withdrawn_df["Shares Offered"] * withdrawn_df["Avg Price"] / 1_000_000
)

print("Non-null value rows:", withdrawn_df["Withdrawn Value"].notna().sum())

withdrawn_df

Non-null value rows: 71


,Symbol,Company Name,Price Range,Shares Offered,Company Class,Avg Price,Withdrawn Value ($m),Withdrawn Value
26,NVL,Novelis Inc.,$18.00 - $21.00,"45,000,000.00",Inc,19.50,877.50,877.50
97,IFIT,iFIT Health & Fitness Inc.,$18.00 - $21.00,"30,769,231.00",Inc,19.50,600.00,600.00
99,HCG,hear.com N.V.,$17.00 - $20.00,"16,220,000.00",Other,18.50,300.07,300.07
41,GGL,Games Global Limited,$16.00 - $19.00,"14,500,000.00",Limited,17.50,253.75,253.75
52,TFG,"The Fortegra Group, Inc.",$15.00 - $18.00,"18,000,000.00",Inc,16.50,297.00,297.00
...,...,...,...,...,...,...,...,...
98,GLGX,"Gerson Lehrman Group, Inc.",-,NaN,Inc,NaN,NaN,NaN
49,COFO,Coforge Limited,-,NaN,Limited,NaN,NaN,NaN
54,VNG,VNG Limited,-,"21,687,082.00",Limited,NaN,NaN,NaN
71,PICS,PicS Ltd.,-,NaN,Limited,NaN,NaN,NaN


In [40]:

# group by company class
withdrawn_df.groupby('Company Class')['Withdrawn Value'].sum()




Company Class
Acq.Corp   4,021.00
Group         33.79
Holdings      75.00
Inc        2,257.16
Limited      549.73
Other        767.92
Name: Withdrawn Value, dtype: float64